In [ ]:
import pandas_gbq
import pandas as pd

In [ ]:
q='''
SELECT is_discount, is_voucher, is_free_delivery, is_qcommerce, is_corporate, value FROM `fulfillment-dwh-production.curated_data_shared_central_dwh.orders` where is_sent = true and global_entity_id="FO_NO" and date(placed_at_local) = "2024-02-12"
'''

In [ ]:
df = pandas_gbq.read_gbq(q)